In [1]:
from datascience import *
from datascience.predicates import are
path_data = '../../../../data/'
import numpy as np
import matplotlib
matplotlib.use('Agg')
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

from urllib.request import urlopen 
import re
from bancor_research.bancor_simulator.v3.spec.network import BancorDapp

# Recall from an earlier chapter that we already defined the whitelisted_tokens as follows.
whitelisted_tokens = {
    "eth": {
        "trading_fee": "1%",
        "bnt_funding_limit": "1000000"
    },
    "link": {
        "trading_fee": "1%",
        "bnt_funding_limit": "1000000"
    },
    "wbtc": {
        "trading_fee": "1%",
        "bnt_funding_limit": "1000000"
    }
}

# There are other possible configuration settings available, however for the present purpose we will use the defaults.
v3 = BancorDapp(whitelisted_tokens=whitelisted_tokens)

v3.create_user('Trader')

v3.set_user_balance(tkn_amt='101', tkn_name='eth', user_name='Alice')
v3.set_user_balance(tkn_amt='101', tkn_name='wbtc', user_name='Bob')
v3.set_user_balance(tkn_amt='10001', tkn_name='link', user_name='Charlie')
v3.set_user_balance(tkn_amt='2000', tkn_name='bnt', user_name='Trader')
v3.set_user_balance(tkn_amt='1', tkn_name='eth', user_name='Trader')
v3.set_user_balance(tkn_amt='3', tkn_name='link', user_name='Trader')

# The actual deposits.

v3.deposit(tkn_amt='100', tkn_name='eth', user_name='Alice')

v3.deposit(tkn_amt='10000', tkn_name='link', user_name='Charlie')

v3.deposit(tkn_amt='100', tkn_name='wbtc', user_name='Bob')

v3.deposit(tkn_amt='1', tkn_name='eth', user_name='Alice')

v3.deposit(tkn_amt='1', tkn_name='link', user_name='Charlie')

v3.deposit(tkn_amt='1', tkn_name='wbtc', user_name='Bob')

for tkn_name in whitelisted_tokens:
    v3.enable_trading(tkn_name)

v3.trade(tkn_amt='2000', source_token='bnt',  target_token='link', user_name='Trader', timestamp=0)

v3.trade(tkn_amt='302.9981', source_token='link',  target_token='bnt', user_name='Trader', timestamp=0)

def read_url(url): 
    return re.sub('\\s+', ' ', urlopen(url).read().decode())

Bootstrap requirements met for eth
Bootstrap requirements met for link
Bootstrap requirements met for wbtc


In [2]:
v3.describe(decimals=6)

bnt           eth          link  \
1 Account Alice                              NaN      0.000000           NaN   
          Bob                                NaN           NaN           NaN   
          Charlie                            NaN           NaN      0.000000   
          Trader                     1977.615297      1.000000      0.001900   
2 Pool    a: TKN Staked Balance              NaN    101.000000  10003.424242   
          b: TKN Trading Liquidity           NaN     20.000000   3336.331433   
          c: BNT Trading Liquidity           NaN  20000.000000  20013.993917   
          d: BNT Current Funding             NaN  20000.000000  20015.980730   
          e: Spot Rate                       NaN   1000.000000      5.998803   
          f: Average Rate                    NaN   1000.000000      6.000000   
          g: Average Inverse Rate            NaN      0.000000      0.000000   
3 Network Master Vault              60022.384703    101.000000  10003.998100   
          Protection Vault              0.000000      0.000000      0.000000   
          Protocol Equity                    NaN           NaN           NaN   
          Rewards Vault                      NaN           NaN           NaN   

                                            wbtc         bnbnt       bneth  \
1 Account Alice                              NaN           NaN  101.000000   
          Bob                           0.000000           NaN         NaN   
          Charlie                            NaN           NaN         NaN   
          Trader                             NaN           NaN         NaN   
2 Pool    a: TKN Staked Balance       101.000000           NaN         NaN   
          b: TKN Trading Liquidity      1.250000           NaN         NaN   
          c: BNT Trading Liquidity  20000.000000           NaN         NaN   
          d: BNT Current Funding    20000.000000           NaN         NaN   
          e: Spot Rate              16000.000000           NaN         NaN   
          f: Average Rate           16000.000000           NaN         NaN   
          g: Average Inverse Rate       0.000000           NaN         NaN   
3 Network Master Vault                101.000000           NaN         NaN   
          Protection Vault              0.000000           NaN         NaN   
          Protocol Equity                    NaN  60000.000000    0.000000   
          Rewards Vault                      NaN      0.000000    0.000000   

                                          bnlink      bnwbtc vbnt  
1 Account Alice                              NaN         NaN  NaN  
          Bob                                NaN  101.000000  NaN  
          Charlie                   10001.000000         NaN  NaN  
          Trader                             NaN         NaN  NaN  
2 Pool    a: TKN Staked Balance              NaN         NaN  NaN  
          b: TKN Trading Liquidity           NaN         NaN  NaN  
          c: BNT Trading Liquidity           NaN         NaN  NaN  
          d: BNT Current Funding             NaN         NaN  NaN  
          e: Spot Rate                       NaN         NaN  NaN  
          f: Average Rate                    NaN         NaN  NaN  
          g: Average Inverse Rate            NaN         NaN  NaN  
3 Network Master Vault                       NaN         NaN  NaN  
          Protection Vault                   NaN         NaN  NaN  
          Protocol Equity               0.000000    0.000000  NaN  
          Rewards Vault                 0.000000    0.000000  NaN

# Trading and Fees - TKN to TKN

Assume now that a trader wishes to exchange 1 ETH for wBTC. From their perspective the process is similar to that
described above. A total of 1 ETH tokens are sent into the vault, and the vault sends 0.055712824 wBTC tokens back, and the change in the vault balances agree with the trader's intuition (as before). However, this situation compounds the effects described in the previous two sections. The changes in the trading liquidity balances, and the accumulation of value to the vortex ledger, can be deduced by considering the two separate legs of the process:

1. Swap 1 ETH for 942.857143 BNT

   - Add 7.619047619 BNT to the staking ledger

   - Add 1.904761905 BNT to the vortex ledger

2. Swap 942.857143 BNT for 0.055712824 wBTC

   - Add 0.000450205 wBTC to the staking ledger

   - Add 1.973681795 BNT to the vortex ledger

In [3]:
v3.trade(tkn_amt='1', source_token='eth',  target_token='wbtc', user_name='Trader', timestamp=0)
v3.describe(decimals=4)

bnt         eth        link  \
1 Account Alice                            NaN      0.0000         NaN   
          Bob                              NaN         NaN         NaN   
          Charlie                          NaN         NaN      0.0000   
          Trader                     1977.6153      0.0000      0.0019   
2 Pool    a: TKN Staked Balance            NaN    101.0000  10003.4242   
          b: TKN Trading Liquidity         NaN     21.0000   3336.3314   
          c: BNT Trading Liquidity         NaN  19055.2381  20013.9939   
          d: BNT Current Funding           NaN  20007.6190  20015.9807   
          e: Spot Rate                     NaN    907.3923      5.9988   
          f: Average Rate                  NaN   1000.0000      6.0000   
          g: Average Inverse Rate          NaN      0.0000      0.0000   
3 Network Master Vault              60022.3847    102.0000  10003.9981   
          Protection Vault              0.0000      0.0000      0.0000   
          Protocol Equity                  NaN         NaN         NaN   
          Rewards Vault                    NaN         NaN         NaN   

                                          wbtc       bnbnt     bneth  \
1 Account Alice                            NaN         NaN  101.0000   
          Bob                           0.0000         NaN       NaN   
          Charlie                          NaN         NaN       NaN   
          Trader                        0.0557         NaN       NaN   
2 Pool    a: TKN Staked Balance       101.0005         NaN       NaN   
          b: TKN Trading Liquidity      1.1943         NaN       NaN   
          c: BNT Trading Liquidity  20940.8835         NaN       NaN   
          d: BNT Current Funding    20000.0000         NaN       NaN   
          e: Spot Rate              17534.2111         NaN       NaN   
          f: Average Rate           16000.0000         NaN       NaN   
          g: Average Inverse Rate       0.0000         NaN       NaN   
3 Network Master Vault                100.9443         NaN       NaN   
          Protection Vault              0.0000         NaN       NaN   
          Protocol Equity                  NaN  60000.0000    0.0000   
          Rewards Vault                    NaN      0.0000    0.0000   

                                        bnlink    bnwbtc vbnt  
1 Account Alice                            NaN       NaN  NaN  
          Bob                              NaN  101.0000  NaN  
          Charlie                   10001.0000       NaN  NaN  
          Trader                           NaN       NaN  NaN  
2 Pool    a: TKN Staked Balance            NaN       NaN  NaN  
          b: TKN Trading Liquidity         NaN       NaN  NaN  
          c: BNT Trading Liquidity         NaN       NaN  NaN  
          d: BNT Current Funding           NaN       NaN  NaN  
          e: Spot Rate                     NaN       NaN  NaN  
          f: Average Rate                  NaN       NaN  NaN  
          g: Average Inverse Rate          NaN       NaN  NaN  
3 Network Master Vault                     NaN       NaN  NaN  
          Protection Vault                 NaN       NaN  NaN  
          Protocol Equity               0.0000    0.0000  NaN  
          Rewards Vault                 0.0000    0.0000  NaN

It is important to realize that there is no BNT transfer, despite both pools reporting a change to the BNT trading liquidity. A virtual trade has been performed, where BNT is still used as numeraire, and which allows for the relative value of ETH and wBTC to be known. Since the liquidity pools are virtual, their relative balance of BNT can be adjusted as though the tokens were sent, when in fact the BNT remains where it was in the vault. Thus, a quasi-single hop trade is achieved.

Therefore, during the trade the following components of the system were changed:

1. The vault balances. The change in the vault is always in agreement with the trader's intuition. In this case, the trader sent 1 ETH into the vault, and received 0.055712824 wBTC from it. Therefore, the vault balance of ETH must have increased by 1 ETH, and decreased by 0.055712824 wBTC.

2. The staking ledger. Trading fees are always taken from the target asset, in this case there are two:

1. The first virtual trade to BNT, resulting in a 7.619047619 increase to the BNT staked balance.

2. The second virtual trade, resulting in a 0.000450205 increase to the wBTC staked balance.

Therefore, the value of the bnBNT pool token, and wBTC pool token must be appreciated by 80% of the fee apparent to the trader (i.e. the total fee - vortex rate).

1. The vortex ledger. Both virtual trades have an apparent effect on the vortex ledger. In the first virtual trade to BNT, one fifth of the fee paid by the trader is collected by the vortex directly; in the second virtual trade, one fifth of the fee in wBTC is swapped back to BNT. Both are added to the BNT balance of the vortex ledger.

2. The trading liquidity. The changes to the trading liquidities on both pools can be understood after accounting for the effect of the collectionby the vortex.

The change in the BNT, ETH, and wBTC trading liquidity can be expressed as follows:

![](https://lh5.googleusercontent.com/doveLoYhNUJtnhVPwl2vneQWuOFLZwtoA1IA-v472nUtYScilTCSe2m1yQXKyquVJjTElyYscYqSSUIjPU9LXrlM88_Un4Lf9dW_aPNWcb7K2uzAgAVMsqQkofmdAb9V8MXfU_Iu)

![](https://lh6.googleusercontent.com/SdYHsyNvZIKz3O7JdQNeu_6XHZp7jK4s_vtuCnm6ekReFAsU4tkR2pGkqzWkneT_jzMs0QR8grLnskRof_oWrKlk8mt49bppggxtYNYibZkq4uBkN0XVvy1azYiBYDVTvA6aR8qz)

![](https://lh5.googleusercontent.com/7SFfNH_BLbsm2nAS_TI4veTsEI4-BmOxw9RypNeEWzaDfA48ELD30aXiopBHIBUSRTjWRAu57rQ1bmH5igSKf6vyR13PS2BSKOx6mM9oqpp7POxr1MWIJLk2v12b7bGSw4iXh_ze)

![](https://lh4.googleusercontent.com/nQbaQymdxqr99dZhRngwd5acVZtmjOr--QqtFH4-OAyTiOkDtl2vynZR50LQVaqv6Xez_9r7yMSPJgq3ayBH2YJK0lpPpFPGYj5vqtIqoAeSF_Y3WSNBcLLbdbJ141nOP-GAIhsA)

![](https://lh3.googleusercontent.com/fB4ZwVR0grJ-ALAIFKwpnR3bgBawzdyk6uKtBUCq1huifwIPIPrDU_vauIEtyXSkETlswyxKYykNJdZtJFDStfqNyXmSMFAzfA6BeY7duBIrCXTiktCX0fRjMJNw6A6FOgSknfSl)

where a1† and a1 are the BNT trading liquidity balances of the source pool (BNT-ETH) after and before the trade,
respectively, b1† and b1 are the TKN trading liquidity balances of the source pool (BNT-ETH) after and before the trade, respectively, a2† and a2 are the BNT trading liquidity balances of the destination pool (BNT-wBTC) after and before the trade, respectively, b2† and b2 are the TKN trading liquidity balances of the destination pool (BNT-wBTC) after and before the trade, respectively, d1 is the source pool fee (BNT-ETH, e.g. 0.01, or 1%), d2 is the destination pool fee (BNT-wBTC, e.g. 0.01, or 1%), and e is the Vortex rate (e.g. 0.2, or 20%). The number of ETH tokens being sent into the vault is x, and the number of wBTC tokens sent back to the trader is tknOut. The change in the ETH trading liquidity (and therefore the update to the available ETH trading liquidity) is unchanged from the standard case. The calculation of the fee awarded to the bnwBTC, and bnBNT pool token holders is:

![](https://lh5.googleusercontent.com/MG169O6uOWvALrOb5TgqaOM-O1Qh96yXNPXFEFluyEH5rxcJFn8B988n0hoF5GbV_UWXwmUF3LXLbwdQLZHG2Zo2VsKvcjnWkkJR1vELWji8hsIOMHt8nAIqZVn0p6hqCfRZd-Vm)

![](https://lh3.googleusercontent.com/HFcLc-pvEdpenFPVNvjVXLL7mGUy5SlK8K78IeFjQ2r5ZlcB-cbh4ah7XtKY8JCvpadLdm1UG1og4eiUXMw85Ou7v6isoCdgjLpa_i5b5ZfwJACKfHgv4k3MTkVDZG7TvW8q4utM)

And the calculation for the fee given to the Bancor Vortex, denominated exclusively in BNT and added to the vortex ledger is:

![](https://lh5.googleusercontent.com/o-GET6e9jIIDlJ54BnkXX_eizMLT8I8g1tV9qIZE88orUlrDTRUa0lxqJVd3uC2InWc682kukPqJoWloUnRzmHP__PXfp_us8_CzQLjmo072dFoik_zSbIVqa3o-FI94lowQznDA)